In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from keras import layers

# !rm -rf ./logs/**

In [3]:
# Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data by flattening & scaling it
x_train = x_train.reshape(-1, 784).astype("float32") / 255
x_test = x_test.reshape(-1, 784).astype("float32") / 255

# Categorical (one hot) encoding of the labels
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [4]:
def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map

In [5]:
RandomFourierFeatures = keras.layers.experimental.RandomFourierFeatures

model1 = keras.Sequential(
    [
        keras.Input(shape=(784,)),
        RandomFourierFeatures(
            output_dim=8192, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=10),
    ]
)
model1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)

model1.fit(x_train, y_train, epochs=50, batch_size=128, validation_split=0.2, callbacks=[
    keras.callbacks.TensorBoard(log_dir="./logs/fit/linear-" + datetime.now().strftime("%Y%m%d-%H%M%S"), histogram_freq=1)
])

Epoch 1/50
375/375 [==============================] - 4s 5ms/step - loss: 0.0839 - acc: 0.8804 - val_loss: 0.0464 - val_acc: 0.9360
Epoch 2/50
375/375 [==============================] - 2s 5ms/step - loss: 0.0414 - acc: 0.9402 - val_loss: 0.0350 - val_acc: 0.9515
Epoch 3/50
375/375 [==============================] - 2s 4ms/step - loss: 0.0319 - acc: 0.9555 - val_loss: 0.0392 - val_acc: 0.9449
Epoch 4/50
375/375 [==============================] - 2s 4ms/step - loss: 0.0269 - acc: 0.9632 - val_loss: 0.0322 - val_acc: 0.9565
Epoch 5/50
375/375 [==============================] - 2s 4ms/step - loss: 0.0235 - acc: 0.9687 - val_loss: 0.0296 - val_acc: 0.9583
Epoch 6/50
375/375 [==============================] - 2s 4ms/step - loss: 0.0230 - acc: 0.9701 - val_loss: 0.0263 - val_acc: 0.9647
Epoch 7/50
375/375 [==============================] - 2s 4ms/step - loss: 0.0205 - acc: 0.9738 - val_loss: 0.0226 - val_acc: 0.9689
Epoch 8/50
375/375 [==============================] - 2s 4ms/step - loss: 0.

In [6]:
def gen_samples(X, psi, t=1000):
    n, _ = X.shape
    return [
        tf.gather_nd(
            indices=tf.expand_dims(tf.random.shuffle(tf.range(n))[:psi], 1), params=X
        ).numpy()
        for _ in range(t)
    ]


t_samples = gen_samples(x_train, psi=16, t=500)

In [7]:
class IsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
      super(IsolationEncodingLayer, self).__init__(**kwargs)
      self.samples = samples
      self.p = p
      self.soft = soft

    def call(self, inputs):
      return _tf_ann(inputs, self.samples, self.p, self.soft)

    def get_config(self):
        config = super().get_config()
        config.update({
            "samples": self.samples,
            "p": self.p,
            "soft": self.soft,
        })
        return config

In [8]:
def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(
            t_samples[i], 
            p = p,
            soft = soft,
            name="ann_{}".format(i)
        )(inputs)
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=10, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


model2 = build_model(t_samples, soft=False)
# model2.summary()

In [9]:
model2.fit(x_train, y_train, epochs=50, batch_size=128, validation_split=0.2, callbacks=[
    keras.callbacks.TensorBoard(log_dir="./logs/fit/hard-" + datetime.now().strftime("%Y%m%d-%H%M%S"), histogram_freq=1)
])

Epoch 1/50
375/375 [==============================] - 576s 1s/step - loss: 0.0504 - acc: 0.9173 - val_loss: 0.0287 - val_acc: 0.9507
Epoch 2/50
375/375 [==============================] - 378s 1s/step - loss: 0.0227 - acc: 0.9638 - val_loss: 0.0234 - val_acc: 0.9625
Epoch 3/50
375/375 [==============================] - 370s 988ms/step - loss: 0.0165 - acc: 0.9762 - val_loss: 0.0217 - val_acc: 0.9665
Epoch 4/50
375/375 [==============================] - 368s 981ms/step - loss: 0.0126 - acc: 0.9832 - val_loss: 0.0211 - val_acc: 0.9687
Epoch 5/50
375/375 [==============================] - 367s 978ms/step - loss: 0.0098 - acc: 0.9878 - val_loss: 0.0207 - val_acc: 0.9677
Epoch 6/50
375/375 [==============================] - 366s 978ms/step - loss: 0.0079 - acc: 0.9905 - val_loss: 0.0203 - val_acc: 0.9706
Epoch 7/50
375/375 [==============================] - 372s 993ms/step - loss: 0.0064 - acc: 0.9931 - val_loss: 0.0200 - val_acc: 0.9705
Epoch 8/50
375/375 [==============================] - 

In [10]:
model3 = build_model(t_samples, soft=True)
# model3.summary()

In [11]:
model3.fit(x_train, y_train, epochs=50, batch_size=128, validation_split=0.2, callbacks=[
    keras.callbacks.TensorBoard(log_dir="./logs/fit/soft-" + datetime.now().strftime("%Y%m%d-%H%M%S"), histogram_freq=1)
])

Epoch 1/50
375/375 [==============================] - 514s 1s/step - loss: 0.1461 - acc: 0.8319 - val_loss: 0.0712 - val_acc: 0.9193
Epoch 2/50
375/375 [==============================] - 289s 771ms/step - loss: 0.0605 - acc: 0.9206 - val_loss: 0.0515 - val_acc: 0.9348
Epoch 3/50
375/375 [==============================] - 288s 769ms/step - loss: 0.0482 - acc: 0.9329 - val_loss: 0.0438 - val_acc: 0.9406
Epoch 4/50
375/375 [==============================] - 285s 761ms/step - loss: 0.0420 - acc: 0.9395 - val_loss: 0.0394 - val_acc: 0.9457
Epoch 5/50
375/375 [==============================] - 288s 769ms/step - loss: 0.0382 - acc: 0.9440 - val_loss: 0.0365 - val_acc: 0.9496
Epoch 6/50
375/375 [==============================] - 293s 781ms/step - loss: 0.0354 - acc: 0.9469 - val_loss: 0.0342 - val_acc: 0.9510
Epoch 7/50
375/375 [==============================] - 295s 787ms/step - loss: 0.0334 - acc: 0.9497 - val_loss: 0.0326 - val_acc: 0.9533
Epoch 8/50
375/375 [==============================]

In [6]:
%tensorboard --logdir ./logs/fit